# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

pd.options.display.max_columns = None

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
SELECT
    'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [4]:
%%sql
CREATE OR REPLACE TABLE competencia_02 AS
SELECT
    *
FROM read_csv_auto('../../../../datasets/raw/competencia_02_crudo.csv')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [5]:
%%sql
SELECT
    *
FROM competencia_02
LIMIT 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,tmobile_app,cmobile_app_trx,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_Finiciomora,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,-2325.76,7273.13,9,1,1,0.0,0.00,2,135427.60,0.0,503316.61,1,778973.10,1,0,0.00,1,71,54777.17,1,53,45966.59,0,0.0,0,0.0,0,0.0,3,103224.0,0.0,0,0.0,0.0,3,128880.26,0,0,0,0,0,1,195311.53,0.0,0,0,0.00,5,7787.38,2,768.70,0,0.00,5,23387.47,0,0.00,0,0.0,0,0.0,0,0.00,18,2501.04,3,0,0.0,3,4989.5,0,0.0,4,34901.44,12,46920.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1,194,0,0,0,0,0,11,45747.0,1,1173.0,336,NaN,NaN,0,0,542893.72,-1429,NaN,45513.70,44899.96,613.73,36679.21,16.41,603215.25,0.0,0.0,8,0.00,-33125.09,0.0,5148,36679.21,30.0,0.0,1806.42,0,0,542893.72,-1429,NaN,46907.58,53401.58,1621.00,36602.13,24.92,603215.25,0.0,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,321.62,2518.96,9,1,1,0.0,27669.01,2,16206.53,0.0,0.00,1,18285.47,2,0,0.00,1,9,32443.22,1,8,29609.15,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.00,0,0,1,0,2,0,0.00,0.0,0,0,0.00,3,25240.27,8,29609.15,0,0.00,0,0.00,0,0.00,0,0.0,0,0.0,1,1199.97,13,14986.76,

In [6]:
%%sql
CREATE OR REPLACE TABLE competencia_02 AS (
    SELECT
        *,
        RANK() OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes DESC) AS rank_foto_mes,
    FROM competencia_02
    )

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [7]:
%%sql
CREATE OR REPLACE TABLE competencia_02 AS (
    SELECT
        *,
        rank_foto_mes*-1 + 1 AS rank_foto_mes_2
    FROM competencia_02
    ORDER BY foto_mes DESC
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [8]:
%%sql
CREATE OR REPLACE TABLE competencia_02 AS (
    SELECT
        *,
        CASE
            WHEN rank_foto_mes_2 = 0 THEN 'BAJA+2'
            WHEN rank_foto_mes_2 =-1 THEN 'BAJA+1'
            ELSE 'CONTINUA'
        END AS clase_ternaria
    FROM competencia_02
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [9]:
%%sql
select
    clase_ternaria,
    count(*) as cant,
    cant/(SELECT count(*) from competencia_02) as prop_tot,
    cant/(SELECT count(*) from competencia_02 WHERE clase_ternaria is not NULL) as prop_m0
from competencia_02
group by clase_ternaria

,clase_ternaria,cant,prop_tot,prop_m0
0,BAJA+1,182285,0.039950,0.039950
1,BAJA+2,184187,0.040367,0.040367
2,CONTINUA,4196338,0.919683,0.919683


In [10]:
%%sql
CREATE OR REPLACE TABLE competencia_02 AS (
    SELECT
        *
    FROM competencia_02
    WHERE
        foto_mes BETWEEN 202001 AND 202105
);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [11]:
%%sql
select
    clase_ternaria,
    count(*) as cant,
    cant/(SELECT count(*) from competencia_02) as prop_tot,
    cant/(SELECT count(*) from competencia_02 WHERE clase_ternaria is not NULL) as prop_m0
from competencia_02
group by clase_ternaria

,clase_ternaria,cant,prop_tot,prop_m0
0,CONTINUA,2637667,0.991741,0.991741
1,BAJA+2,10870,0.004087,0.004087
2,BAJA+1,11096,0.004172,0.004172


In [12]:
%%sql
SELECT
    foto_mes,
    count(*) as cant
FROM competencia_02
WHERE clase_ternaria = 'BAJA+2'
GROUP BY foto_mes

,foto_mes,cant
0,202001,628
1,202002,540
2,202003,217
3,202004,375
4,202005,509
5,202006,644
6,202007,641
7,202008,541
8,202009,471
9,202010,553


In [13]:
%%sql
COPY competencia_02
TO '../../../../datasets/processed/competencia_02_clase_ternaria_rank.parquet' (FORMAT PARQUET);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
